# Параллельная разработка с библиотекой `concurrent.futures`

## Основы

Логично предположить, что для параллельной разработки на Python созданы какие-то библиотеки. Первой из них была `concurrent.futures`. Библиотека эта введена в Python начиная с версии 3.2, однако позже она была адаптирована для Python начиная с версии 2.5. Основной подход в работе с этой библиотекой воплощает следующую мысль Микеле Симионато (Michele Simionato):

_Люди, критикующие явную работу с несколькими потоками, обычно являются системными программистами, которые имеют в виду варианты использования, с которыми типичный прикладной программист никогда не столкнется в своей жизни. […] В 99% случаев прикладному программисту достаточно создавать группы независимых потоков и собирать результаты их работы в очереди_.

Далее мы рассмотрим такую сущность, как футуры. Если говорить кратко, то футура - это объект, предоставляющий возможность асинхронного выполнения операции. 

Начнем с простого примера. Допустим, нам надо написать программу, скачивающую с интернета какие-то файлы. Есть 2 подхода: либо мы качаем файлы последовательно, либо параллельно. Рассмотрим оба варианта реализации.

In [1]:
%%writefile flags_seq.py

import os
import time
import sys
import shutil
import requests

countries = 'CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR'.split()
base_url = 'http://flupy.org/data/flags'
dest_dir = './downloads'

def save_image(img, filename):
    path = os.path.join(dest_dir, filename)
    with open(path, 'wb') as fp:
        fp.write(img)

def show(text):
    #Печатаем без перехода на следующую строку
    print(f'{text}', end=' ')

def get_flag(country):
    url = f'{base_url}/{country}/{country}.gif'
    response = requests.get(url)
    return response.content

def init_dir(dest_dir):
    print('Initialization of download directory...')
    if os.path.exists(dest_dir):        
        shutil.rmtree(dest_dir)
    os.mkdir(dest_dir)
    print('Done.')
    
def download_flag(country):
    image = get_flag(country.lower())
    save_image(image, f'{country.lower()}.gif')
    show(country)
    return country

def download_flags(country_lst):
    print('Downloading flags...')
    downloaded = [download_flag(country) for country in sorted(country_lst)]
    print('Done.')
    return len(downloaded)

def main(downloader):
    init_dir(dest_dir)
    start_time = time.time()
    count = downloader(countries)
    time_taken = time.time() - start_time
    print(f'\n{count} flag(s) downloaded in {time_taken:.2f} seconds.')

if __name__ == '__main__':
    main(download_flags)

Overwriting flags_seq.py


In [2]:
%%bash

python3 flags_seq.py

Initialization of download directory...
Done.
BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN Done.

20 flag(s) downloaded in 14.50 seconds.


Если качать последовательно, то на скачивание 20 маленьких `*.gif`-файлов уходит от 10 до 20 секунд. Посмотрим, сколько займет та же операция при использовании библиотеки `concurrent.futures`. Также, чтобы не переписывать код повторно, мы используем некоторые функции из написанного выше скрипта, выполняющего последовательное скачивание.

In [3]:
%%writefile flags_fut.py

from concurrent import futures
from flags_seq import main, download_flag

max_workers = 20

def download_flags(country_lst):
    worker_number = min(max_workers, len(country_lst))
    with futures.ThreadPoolExecutor(worker_number) as executor:
        downloaded_flags = executor.map(download_flag, sorted(country_lst))
    return len(list(downloaded_flags))

if __name__ == '__main__':
    main(download_flags)

Overwriting flags_fut.py


In [4]:
%%bash

python3 flags_fut.py

Initialization of download directory...
Done.
BR DE US CN CD EG NG PK MX TR PH JP RU FR IN ET ID IR BD VN 
20 flag(s) downloaded in 1.03 seconds.


В данном случае на скачивание всех флагов уходит не более 3 секунд.

Код выше явным образом футуры не использует, оставляя их под капотом. По факту футуры - это объекты, инкапсулирующие операции, которые могут быть выполнены аснхронно. Есть 2 класса футур - `concurrent.futures.Future` и `asyncio.Future`. Они, естественно, несколько различаются. У них есть следующие интересные методы:

* `.done()` - отвечает на вопрос, отработала ли футура  
* `.add_done_callback()` - назначает функцию, которая будет вызвана по завершении работы футуры  
* `.result()` - выдает результат работы футуры  

Мы можем попытаться более явно использовать футуры, 

In [5]:
%%writefile flags_fut2.py

from concurrent import futures
from flags_seq import main, download_flag

max_workers = 20

def download_flags(country_lst):
    worker_number = min(max_workers, len(country_lst))
    results = []
    with futures.ThreadPoolExecutor(worker_number) as executor:
        to_do = [executor.submit(download_flag, country) for country in country_lst]        
        for future in futures.as_completed(to_do):
            result = future.result()
            print(f'{future} result: {result}')
            results.append(result)
    return len(results)
    #    downloaded_flags = executor.map(download_flag, sorted(country_lst))
    #return len(list(downloaded_flags))

if __name__ == '__main__':
    main(download_flags)

Overwriting flags_fut2.py


In [6]:
%%bash

python3 flags_fut2.py

Initialization of download directory...
Done.
BR <Future at 0x7f2421d3ed30 state=finished returned str> result: BR
CN <Future at 0x7f2421d26130 state=finished returned str> result: CN
ID <Future at 0x7f2421d3e400 state=finished returned str> result: ID
CD <Future at 0x7f2420461430 state=finished returned str> result: CD
US <Future at 0x7f2421d31a90 state=finished returned str> result: US
RU <Future at 0x7f24204a8250 state=finished returned str> result: RU
EG <Future at 0x7f24204c4be0 state=finished returned str> result: EG
DE <Future at 0x7f24204ce550 state=finished returned str> result: DE
IR <Future at 0x7f24204cee80 state=finished returned str> result: IR
ET <Future at 0x7f24204c42e0 state=finished returned str> result: ET
BD <Future at 0x7f24204a0940 state=finished returned str> result: BD
MX <Future at 0x7f24204b1430 state=finished returned str> result: MX
PK <Future at 0x7f2421d486a0 state=finished returned str> result: PK
NG <Future at 0x7f2421d31730 state=finished returned str>

Как известно, в Python есть такой компонент, как Global Interpreter Lock (GIL), который блокирует одновременное использование вычислительных ресурсов несколькими потоками. При этом, как мы видели выше, многопоточный код скачал файлы существенно быстрее, чем однопоточный. Как такое произошло? Очевидно, что пользовательский код Python может как что-то считать сам, так и вызывать функции библиотек. Библиотеки могут тоже, как считать, так и вызывать другие библиотеки. Здесь важно вспомнить, что вызовы библиотек в конечном счете могут превращаться в вызовы API операционной системы, которые GIL не блокирует. Скачивание файла - это пример вызова API операционной системы, который GIL не блокирует. Поэтому и получается сильно быстрее. Если обобщить сказанное, то получается, что все операции ввода-вывода в Python хорошо параллелятся.